# DuckDB workshop

[Parquet](https://parquet.apache.org/) is a faster, more compact, and better typed storage format than CSV, Excel, JSON, or SQLite. Prefer it for data storage.

[DuckDB](https://duckdb.org/) is faster than databases and as portable as SQLite. Prefer it for local large data analysis.

In [1]:
!pip install duckdb pandas sqlalchemy sqlite3

In [2]:
# If this doesn't work, you can download the file manually from the link below
!curl https://github.com/plotly/datasets/raw/master/2015_flights.parquet --output 2015_flights.parquet --location --continue-at - --silent

In [3]:
# Take a look at the first few rows of the dataset
import pandas as pd

df = pd.read_parquet('2015_flights.parquet')
df.head()

,DEPARTURE_DELAY,ARRIVAL_DELAY,DISTANCE,SCHEDULED_DEPARTURE
0,-11.0,-22.0,1448,0.083333
1,-8.0,-9.0,2330,0.166667
2,-2.0,5.0,2296,0.333333
3,-5.0,-9.0,2342,0.333333
4,-1.0,-21.0,1448,0.416667


## Parquet is faster and smaller

Let's save this in a few other formats and look at their speed and sizes

In [4]:
%timeit -n 1 -r 1 df.to_csv('2015_flights.csv', index=False)

7.62 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [5]:
%timeit -n 1 -r 1 df.to_json('2015_flights.json', orient='records')

3.39 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [6]:
%timeit -n 1 -r 1 df.to_sql('2015_flights', 'sqlite:///2015_flights.db', index=False, if_exists='replace')

15.3 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [7]:
%timeit -n 1 -r 1 df.to_parquet('2015_flights.saved.parquet')

411 ms ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [8]:
# If this command doesn't work, just look at the file size in this directory
!ls -la 2015_flights.*

-rwxrwxr-x+ 1 Anand None 166305456 Mar 28 22:37 2015_flights.csv
-rwxrwxr-x+ 1 Anand None 133074944 Mar 28 22:37 2015_flights.db
-rwxrwxr-x+ 1 Anand None 545177083 Mar 28 22:37 2015_flights.json
-rwxrwxr-x+ 1 Anand None  25238218 Mar 16 08:38 2015_flights.parquet
-rwxrwxr-x+ 1 Anand None  25238340 Mar 28 22:37 2015_flights.saved.parquet


# DuckDB is faster and takes less memory than Pandas

This is because:

1. DuckDB uses **parallel processing**. Pandas cannot.
2. DuckDB uses **columnar storage**, which is efficient for analysis. Pandas uses row-based storage.
3. DuckDB uses **on-disk operations**. Pandas loads everything into memory.

Let's calculate the number of unique routes that were delayed by 1 hour, 2 hours, etc.

This is a hard problem because

1. There are several delay buckets (1, 2, 3, .. 1988)
2. Unique count is a slow operation

In [9]:
# This is the result we want to achieve
delays = df.groupby('DEPARTURE_DELAY')['DISTANCE'].nunique()
delays[delays.index > 0]

DEPARTURE_DELAY
1.0       1308
2.0       1300
3.0       1296
4.0       1289
5.0       1294
          ... 
1631.0       1
1649.0       1
1670.0       1
1878.0       1
1988.0       1
Name: DISTANCE, Length: 1162, dtype: int64

In [10]:
# This takes ~5 seconds in Pandas
%timeit df.groupby('DEPARTURE_DELAY')['DISTANCE'].nunique()

5.4 s ± 304 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [11]:
# Let's do the same thing with DuckDB
import duckdb

delays = duckdb.query('''SELECT DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) FROM "2015_flights.parquet" GROUP BY DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY''').df()
delays[delays['DEPARTURE_DELAY'] > 0]

,DEPARTURE_DELAY,count(DISTINCT DISTANCE)
55,1.0,1308
56,2.0,1300
57,3.0,1296
58,4.0,1289
59,5.0,1294
...,...,...
1212,1631.0,1
1213,1649.0,1
1214,1670.0,1
1215,1878.0,1


In [12]:
# This takes under 200ms in DuckDB
%timeit duckdb.sql('''SELECT DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) FROM "2015_flights.parquet" GROUP BY DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY''').df()

160 ms ± 7.49 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


The entire process runs on disk, not in memory.

In [13]:
conn = duckdb.connect(database=':memory:', read_only=False)
delays = conn.execute('SELECT DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) FROM "2015_flights.parquet" GROUP BY DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY').df()
print(conn.execute("PRAGMA database_size").fetchall())

[('memory', '0 bytes', 0, 0, 0, 0, '0 bytes', '0 bytes', '50.0 GiB')]


# You can mix it up with Pandas

DuckDB can also directly query Pandas DataFrames. You can use it as a replacement for any specific slow Pandas operation.

In [14]:
df = pd.read_parquet('2015_flights.parquet')
delays = duckdb.sql('SELECT DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) FROM df GROUP BY DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY').df()

In [15]:
# This takes ~25ms -- 20x faster than Pandas
%timeit delays = duckdb.sql('''SELECT DEPARTURE_DELAY, COUNT(DISTINCT DISTANCE) FROM df GROUP BY DEPARTURE_DELAY ORDER BY DEPARTURE_DELAY''').df()

25.9 ms ± 2.06 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [16]:
# You can interleave Pandas and DuckDB operations

# Step 1: Segment Flights into Short, Medium, Long
distance_segmented_delays = duckdb.query("""
SELECT
  CASE
    WHEN DISTANCE < 1000 THEN 'Short'
    WHEN DISTANCE >= 1000 AND DISTANCE < 2000 THEN 'Medium'
    ELSE 'Long'
  END AS Distance_Category,
  DEPARTURE_DELAY,
  ARRIVAL_DELAY
FROM df
""").to_df()

# Step 2: Correlate between Departure and Arrival Delays
correlations = (
    distance_segmented_delays.groupby('Distance_Category')
    .apply(lambda x: x[['DEPARTURE_DELAY', 'ARRIVAL_DELAY']].corr().iloc[0, 1])
    .reset_index(name='Correlation')
)

# Step 3: Calculate Average Correlation
duckdb.query("SELECT AVG(Correlation) AS Avg_Correlation FROM correlations").to_df()

,Avg_Correlation
0,0.936172


# Exercise

Write the following queries in both Pandas as well as DuckDB. Note the performance of both.

**Exercise 1**: Rank the `ARRIVAL_DELAY` for each `DISTANCE`. (Prioritize rows with lower `DEPARTURE_DELAY` in case of ties.) Get rows with rank 1. (You should get ~13xx rows)

**Exercise 2**: (This is independent of Exercise 1.) Run the code below to create `cost.parquet`. This shows the cost of each arrival delay or departure delay. Calculate the total cost of delays in `2015_flights.parquet` (You should get ~600 mn)

In [17]:
# Code to create `cost.parquet`

import numpy as np

np.random.seed(0)
arrival_delays, departure_delays = df['ARRIVAL_DELAY'].unique(), df['DEPARTURE_DELAY'].unique()
cost = pd.DataFrame({
  'type': ['ARRIVAL_DELAY'] * len(arrival_delays) + ['DEPARTURE_DELAY'] * len(departure_delays),
  'delay': np.concatenate([arrival_delays, departure_delays]),
  'cost': np.random.randint(0, 100, len(arrival_delays) + len(departure_delays))
})
cost.to_parquet('cost.parquet')
cost

,type,delay,cost
0,ARRIVAL_DELAY,-22.0,44
1,ARRIVAL_DELAY,-9.0,47
2,ARRIVAL_DELAY,5.0,64
3,ARRIVAL_DELAY,-21.0,67
4,ARRIVAL_DELAY,8.0,67
...,...,...,...
2454,DEPARTURE_DELAY,999.0,38
2455,DEPARTURE_DELAY,1378.0,68
2456,DEPARTURE_DELAY,888.0,39
2457,DEPARTURE_DELAY,1194.0,79
